In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 1. Camada Bronze: Ingestão dos Dados de Filmes
# MAGIC 
# MAGIC **Objetivo:** Ler o arquivo CSV bruto da Landing Zone e salvá-lo como uma tabela Delta na camada Bronze, adicionando metadados de controle para rastreabilidade.

# COMMAND ----------

# Importa as funções necessárias do PySpark
from pyspark.sql.functions import current_timestamp, input_file_name

# COMMAND ----------

# Define o catálogo que será utilizado durante a execução
spark.sql("USE CATALOG movies_catalog")

# COMMAND ----------

# MAGIC %md
# MAGIC ### Definindo os Caminhos de Origem e Destino

# COMMAND ----------

# O caminho completo para o arquivo CSV dentro do Volume na camada de Landing
source_file_path = "/Volumes/movies_catalog/landing/landing_zone_movies/top_100_movies_full_best_effort.csv"

# Nome da tabela de destino na camada Bronze
bronze_table_name = "bronze.movies_raw"

# COMMAND ----------

# MAGIC %md
# MAGIC ### Leitura, Transformação e Carga

# COMMAND ----------

# Tenta executar o processo de leitura e gravação
try:
    # Passo 1: Leitura do arquivo CSV bruto da Landing Zone
    # inferSchema deduz os tipos de dados das colunas
    # header=true indica que a primeira linha do arquivo é o cabeçalho
    df_raw = spark.read.format("csv") \
                       .option("header", "true") \
                       .option("inferSchema", "true") \
                       .load(source_file_path)

    # Passo 2: Adição de metadados de controle
    #   - data_ingestao: registra quando o dado foi processado
    #   - arquivo_origem: registra de qual arquivo o dado veio
    df_bronze = df_raw.withColumn("data_ingestao", current_timestamp()) \
                      .withColumn("arquivo_origem", input_file_name())

    # Passo 3: Salva o DataFrame como uma tabela Delta na camada Bronze
    #   - mode("overwrite"): apaga os dados existentes e os substitui pelos novos
    #   - option("overwriteSchema", "true"): permite que a estrutura da tabela seja atualizada se houver mudanças
    df_bronze.write.format("delta") \
             .mode("overwrite") \
             .option("overwriteSchema", "true") \
             .saveAsTable(bronze_table_name)
    
    print(f"SUCESSO: A tabela '{bronze_table_name}' foi criada/atualizada na camada Bronze.")

except Exception as e:
    # Informa o erro caso o processo falhe
    print(f"ERRO: Não foi possível processar o arquivo. Verifique o caminho e a existência do arquivo.")
    print(f"Detalhes do erro: {e}")
    # Levanta o erro para que o Job do Databricks registre a falha
    raise e

# COMMAND ----------

# MAGIC %md
# MAGIC ### Verificação dos Dados na Camada Bronze

# COMMAND ----------

# Exibe as 10 primeiras linhas da tabela recém-criada para verificação
display(spark.table(bronze_table_name).limit(10))